In [0]:
# MAGIC %md
# MAGIC # 🧹 03_Silver_Cleaning
# MAGIC Limpia, enriquece, valida y unifica los datos de transacciones QR desde todas las fuentes Bronze.
# MAGIC - Lee desde `bronze.qr_transactions_raw` (sintético) y `bronze.qr_transactions_kaggle_raw` (Kaggle).
# MAGIC - **Unifica** los dos datasets en uno solo.
# MAGIC - Limpia, aplica hashing a PII y enriquece los datos.
# MAGIC - Valida la calidad de los datos combinados.
# MAGIC - Escribe en la tabla `silver.qr_transactions`.

# COMMAND ----------

from pyspark.sql.functions import sha2, col, to_date, current_timestamp, lit

# --- 1. Configuración de Rutas y Tablas ---
# CAMBIO CLAVE: Definimos ambas tablas de origen
bronze_synthetic_table = "fraude_qr.bronze.qr_transactions_raw"
bronze_kaggle_table = "fraude_qr.bronze.qr_transactions_kaggle_raw"

# La tabla Silver de destino será nuestra única fuente de verdad
silver_table = "fraude_qr.silver.qr_transactions"

# --- 2. Leer desde ambas fuentes Bronze ---
print(f"📂 Leyendo datos sintéticos desde: {bronze_synthetic_table}")
df_synthetic = spark.table(bronze_synthetic_table)

print(f"📂 Leyendo datos de Kaggle desde: {bronze_kaggle_table}")
df_kaggle = spark.table(bronze_kaggle_table)

# --- 3. Unificar los DataFrames ---
print("\n🔗 Unificando los datasets de Bronze...")

# Usamos unionByName para unir por nombre de columna, lo que es más robusto.
# allowMissingColumns=True añade columnas que falten en uno de los DFs como nulas.
df_combined = df_synthetic.unionByName(df_kaggle, allowMissingColumns=True)

print(f"📊 Registros totales combinados: {df_combined.count():,}")

# --- 4. Aplicar Transformaciones y Limpieza ---
print("🧼 Aplicando limpieza y transformaciones al dataset unificado...")

df_clean = (
    df_combined
    # 1. Eliminar columnas residuales si existen (de Pandas)
    .drop("__index_level_0__")
    
    # 2. Asegurar columna de partición 'transaction_date' a partir de 'created_at'
    # Esta es la columna que definimos en nuestro DDL de Silver
    .withColumn("transaction_date", to_date(col("created_at")))
    
    # 3. Aplicar hashing a columnas con posible PII
    # El hashing se aplica de forma consistente a ambos datasets
    .withColumn("device_id_hashed", sha2(col("device_id"), 256))
    .withColumn("qr_hash_hashed", sha2(col("qr_hash"), 256))
    
    # 4. Agregar metadata del pipeline
    .withColumn("processed_at", current_timestamp())
    
    # 5. Eliminar columnas sensibles originales y la columna 'date' redundante
    .drop("device_id", "qr_hash", "date")
)

# --- 5. Validar Calidad de los Datos Combinados ---
print("✅ Aplicando validaciones de calidad...")

# Aplicamos las mismas reglas de calidad a todos los datos
# Esto asegura la integridad de nuestra capa Silver
initial_count = df_clean.count()

df_validated = df_clean.filter(
    (col("tx_id").isNotNull()) &
    (col("created_at").isNotNull()) &
    (col("amount") > 0) &
    (col("merchant_id").isNotNull()) &
    (col("payer_id").isNotNull()) &
    (col("is_fraud").isin([0, 1]))
)

final_count = df_validated.count()
print(f"📉 Registros eliminados por validaciones: {initial_count - final_count:,}")
print(f"📊 Registros válidos finales: {final_count:,}")

# --- 6. Escribir en Tabla Silver ---
print(f"\n💾 Escribiendo en tabla Silver: {silver_table}")

(
    df_validated
    .write
    .mode("overwrite")
    .partitionBy("transaction_date") # Particionamos por la columna correcta
    .option("overwriteSchema", "true")
    .saveAsTable(silver_table)
)

print("🎉 ¡Capa Silver creada y validada exitosamente!")

# --- 7. Verificar Resultados ---
print(f"\n🔍 Muestra de datos en {silver_table}:")
spark.table(silver_table).limit(5).display()

📂 Leyendo datos sintéticos desde: fraude_qr.bronze.qr_transactions_raw
📂 Leyendo datos de Kaggle desde: fraude_qr.bronze.qr_transactions_kaggle_raw

🔗 Unificando los datasets de Bronze...
📊 Registros totales combinados: 7,362,620
🧼 Aplicando limpieza y transformaciones al dataset unificado...
✅ Aplicando validaciones de calidad...
📉 Registros eliminados por validaciones: 16
📊 Registros válidos finales: 7,362,604

💾 Escribiendo en tabla Silver: fraude_qr.silver.qr_transactions
🎉 ¡Capa Silver creada y validada exitosamente!

🔍 Muestra de datos en fraude_qr.silver.qr_transactions:


tx_id,created_at,merchant_id,payer_id,amount,currency,qr_type,mcc,payer_lat,payer_lon,merchant_lat,merchant_lon,channel,has_error,error_code,is_fraud,source_file,ingested_at,bronze_batch_id,transaction_date,device_id_hashed,qr_hash_hashed,processed_at
72413811,1970-01-04T00:00:00.000Z,265,13812,2947783.87,USD,dynamic,3360,-35.311,-59.211,-34.236,-58.135999999999996,app,0,null,1,null,null,null,1970-01-04,d893b38a514083ebb3f61241ff43aa15338224644ad5a63ae42e109c3fda2d5b,0558a9b3b1c74d0f7fc0a9fac8c60850caac50ecf8ac84ce94921d15dc7af746,2025-09-20T00:34:16.534Z
72934100,1970-01-04T00:00:00.000Z,946,34101,2947783.87,USD,dynamic,5567,-34.6,-58.5,-33.555,-57.455,app,0,null,1,null,null,null,1970-01-04,d893b38a514083ebb3f61241ff43aa15338224644ad5a63ae42e109c3fda2d5b,0558a9b3b1c74d0f7fc0a9fac8c60850caac50ecf8ac84ce94921d15dc7af746,2025-09-20T00:34:16.534Z
72831591,1970-01-04T00:00:00.000Z,3010,31592,730574.39,USD,dynamic,3360,-35.091,-58.991,-34.491,-58.391,app,0,null,1,null,null,null,1970-01-04,d893b38a514083ebb3f61241ff43aa15338224644ad5a63ae42e109c3fda2d5b,0558a9b3b1c74d0f7fc0a9fac8c60850caac50ecf8ac84ce94921d15dc7af746,2025-09-20T00:34:16.534Z
72895449,1970-01-04T00:00:00.000Z,1848,45450,730574.39,USD,dynamic,5567,-34.949,-58.849,-33.653,-57.553,app,0,null,1,null,null,null,1970-01-04,d893b38a514083ebb3f61241ff43aa15338224644ad5a63ae42e109c3fda2d5b,0558a9b3b1c74d0f7fc0a9fac8c60850caac50ecf8ac84ce94921d15dc7af746,2025-09-20T00:34:16.534Z
72726957,1970-01-04T00:00:00.000Z,7196,26958,3786438.03,USD,dynamic,3360,-35.457,-59.357,-34.305,-58.205,app,0,null,1,null,null,null,1970-01-04,d893b38a514083ebb3f61241ff43aa15338224644ad5a63ae42e109c3fda2d5b,0558a9b3b1c74d0f7fc0a9fac8c60850caac50ecf8ac84ce94921d15dc7af746,2025-09-20T00:34:16.534Z
